In [1]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import tensorflow as tf
import pickle
import sys
import yaml

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, TimeDistributed, Bidirectional
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.utils import plot_model
from keras import optimizers
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import load_model
from sklearn import preprocessing
from copy import deepcopy

import datetime
import math
import hashlib
import time
import os
from datagenerator import DataGenerator

# add
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used
# add
config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
# # sess = tf.Session(config=config) 
sess = tf.compat.v1.Session(config=config) 

def loadConfig():
#     with open('/home/wesleyjtann/ddos_research/test/light_config_b.yaml', "r") as ymlfile:
    with open('./light_config.yaml', "r") as ymlfile: #GE, hulk, slowhttp, slowloris, heartbleed
        cfg = yaml.load(ymlfile)
    return cfg

config = loadConfig()
# artefact = 'artefact' #'artefact_b' #

Using TensorFlow backend.
/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/we

In [2]:
df_test = pd.read_csv(config['metadata']['uniqueID'] + '/artefact/TEST.csv')
df_test

,Source IP,Dest IP,Protocols,Info,Input,Histo
0,115.110.73.128,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xa9ee, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0xa9ee, seq=1280/5, tt..."
1,115.110.73.128,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xa9ee, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0xa9ee, seq=1280/5, tt..."
2,115.110.73.128,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xa9ee, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0xa9ee, seq=1280/5, tt..."
3,115.110.73.128,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xa9ee, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0xa9ee, seq=1280/5, tt..."
4,115.110.73.128,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0xa9ee, seq=1280/5, tt...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0xa9ee, seq=1280/5, tt..."
...,...,...,...,...,...,...
532929,98.5.101.196,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x1212, seq=10/2560, t...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0x1212, seq=10/2560, t..."
532930,98.5.101.196,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x1212, seq=10/2560, t...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0x1212, seq=10/2560, t..."
532931,98.5.101.196,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x1212, seq=11/2816, t...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0x1212, seq=11/2816, t..."
532932,98.5.101.196,71.126.222.64,raw:ip:icmp,"Echo (ping) request id=0x1212, seq=11/2816, t...",2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SEP>2<SE...,"Echo (ping) request id=0x1212, seq=11/2816, t..."


In [8]:
df_norm = pd.read_csv(config['metadata']['uniqueID'] + '/artefact/N1_original.csv')

print(len(df_norm))
df_norm.head()

37264


,Source IP,Dest IP,Protocols,Info,Input,Histo
0,126.120.0.39,71.126.222.64,raw:ip:tcp,"80 → 46689 [SYN, ACK] Seq=0 Ack=1 Win=65535 Le...",117<SEP>45<SEP>117<SEP>49<SEP>117<SEP>101<SEP>...,"80 → 46689 [SYN, ACK] Seq=0 Ack=1 Win=65535 Le..."
1,126.120.0.39,71.126.222.64,raw:ip:tcp,"80 → 46689 [PSH, ACK] Seq=13049 Ack=123 Win=65...",168<SEP>25<SEP>168<SEP>25<SEP>168<SEP>25<SEP>1...,"80 → 46689 [PSH, ACK] Seq=13049 Ack=123 Win=65..."
2,126.120.0.39,71.126.222.64,raw:ip:tcp,"80 → 46689 [PSH, ACK] Seq=27385 Ack=123 Win=65...",124<SEP>124<SEP>168<SEP>25<SEP>168<SEP>25<SEP>...,"80 → 46689 [PSH, ACK] Seq=27385 Ack=123 Win=65..."
3,126.120.0.39,71.126.222.64,raw:ip:tcp,"80 → 46689 [PSH, ACK] Seq=40097 Ack=123 Win=65...",25<SEP>124<SEP>168<SEP>25<SEP>114<SEP>114<SEP>...,"80 → 46689 [PSH, ACK] Seq=40097 Ack=123 Win=65..."
4,126.120.0.39,71.126.222.64,raw:ip:tcp,"80 → 46690 [PSH, ACK] Seq=18593 Ack=122 Win=65...",118<SEP>45<SEP>224<SEP>224<SEP>224<SEP>224<SEP...,"80 → 46690 [PSH, ACK] Seq=18593 Ack=122 Win=65..."


In [9]:
df_norm = df_norm.sample(frac=1)

df_normN1 = df_norm[:35000] #[:35263]
df_normN2 = df_norm[35001:] #[35264:]

## Save new N1
# df_normN1.to_csv('../1_testmimic_caida07/ids_test2/artefact/N1_new.csv',index=False)

In [10]:
len(df_normN2)

2263

In [11]:
## True normals

# df_norm = df_test[(df_test['Attack']==0)]
# df_norm = df_test

df_normN2['Attack']=0

print("length df_norm: ",  len(df_normN2)) 

length df_norm:  2263


/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df_normN2.to_csv('../1_testmimic_caida07/ids_test2/artefact/TESTnorm.csv', index=False)

In [ ]:
3262/.2